In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import math
from pprint import pprint

In [3]:
# Load the Zip CSV
zip_csv = "./Resources/ZipCode.csv"
zip_df = pd.read_csv(zip_csv)
zip_df

,Zip Code,City,County
0,94203,Sacramento,Sacramento County
1,94204,Sacramento,Sacramento County
2,94205,Sacramento,Sacramento County
3,94206,Sacramento,Sacramento County
4,94207,Sacramento,Sacramento County
...,...,...,...
127,95853,Sacramento,Sacramento County
128,95860,Sacramento,Sacramento County
129,95864,Sacramento,Sacramento County
130,95865,Sacramento,Sacramento County


In [5]:
# Load the Crime CSV
crime_csv = "./Resources/Crime.csv"
crime_df = pd.read_csv(crime_csv)
crime_df

,X,Y,FID,ActivityNumber,District,Neighborhood,OccurenceStartDate,OccurenceEndDate,ReportDate,OccurenceLocation,OccurenceCity,OccurenceZipCode,PrimaryViolation
0,6.765513e+06,1.994405e+06,1,NaN,NaN,Old Fair Oaks,2009/03/26 17:10:00,2009/03/26 17:19:59,2009/03/26 17:39:00,9300 Block of Fair O,Fair Oaks,95628,PC 594(B)(1) Vandalism ($400 Or More)
1,1.517236e-04,-2.607703e-06,2,"]\=""-0092335",North Central,Carmichael Park,2018/03/19 21:30:00,2018/03/19 21:59:59,2018/03/20 17:00:00,5100 Block of Janell,Carmichael,95608,PC 273.5 Inflict Crpl Inj Sp/Cohab
2,6.756477e+06,1.974923e+06,3,03/30-0055028,East,Cordova Meadows,2015/03/06 01:11:00,2015/03/06 01:11:00,2015/03/06 00:00:00,10000 Block of Terra,Rancho Cordova,95670,PC 594(B)(2)(A) Vandalism Less Than $400
3,6.741558e+06,1.960069e+06,4,19-0009846,East,Rosemont Downs,2019/01/09 08:52:59,2019/01/09 09:04:00,2019/01/09 14:19:00,8900 Block of Canberra Dr,Sacramento,95826,PC 459 Burglary
4,6.749211e+06,2.004272e+06,5,1977-0020606,North,Foothill Farms NE,1977/01/01 00:00:00,2001/01/01 00:00:00,2019/08/06 00:00:00,5200 Block of Auburn Blvd,Sacramento,95841,PC 261(A)(1) Rape:Victim Incapable Of Giving C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362000,1.517236e-04,-2.607703e-06,362001,2020-8002056,NaN,NaN,2018/12/15 02:16:00,2019/09/23 16:18:00,2020/01/28 03:24:59,5100 Block of spyglass drive DR,las vegas nevada,NaN,PC 530.5(B) Using Another Person's Identifying...
362001,1.517236e-04,-2.607703e-06,362002,2020-8002082,NaN,NaN,2019/11/20 01:01:00,2019/11/20 23:58:59,2020/01/10 10:44:00,8300 Block of MEDEIROS WAY,SACRAMENTO,95829,PC 368(D) Theft of Elder
362002,6.755575e+06,2.005506e+06,362003,2020-8002107,North,Mercy San Juan,2020/01/21 03:00:00,2020/01/21 03:14:59,2020/01/21 10:28:00,6100 Block of Marwick Way,Carmichael,95608,PC 487 Theft from vehicle > $950
362003,6.764044e+06,1.997386e+06,362004,2-0282708,Northeast,Old Fair Oaks,2012/12/20 13:57:00,2012/12/20 13:57:00,2012/12/20 13:57:00,7400 Block of Windin,Fair Oaks,95628,PC 459 Burglary


In [11]:
# Get a list of the indexes where the OccuranceZipCode is empty or NaN
crime_zip_eq_null_index = crime_df.index[(crime_df["OccurenceZipCode"] == "") | 
                                         (crime_df["OccurenceZipCode"].isnull()) ]
crime_zip_eq_null_index

Int64Index([     9,     55,     63,     64,     70,     78,    106,    221,
               225,    376,
            ...
            360576, 360580, 360711, 361257, 361315, 361496, 361548, 361636,
            361894, 362000],
           dtype='int64', length=2609)

In [12]:
# Drop the indexes where the OccuranceZipCode is empty or NaN
preclean_crime_df = crime_df.drop(crime_zip_eq_null_index, inplace=False)
preclean_crime_df

,X,Y,FID,ActivityNumber,District,Neighborhood,OccurenceStartDate,OccurenceEndDate,ReportDate,OccurenceLocation,OccurenceCity,OccurenceZipCode,PrimaryViolation
0,6.765513e+06,1.994405e+06,1,NaN,NaN,Old Fair Oaks,2009/03/26 17:10:00,2009/03/26 17:19:59,2009/03/26 17:39:00,9300 Block of Fair O,Fair Oaks,95628,PC 594(B)(1) Vandalism ($400 Or More)
1,1.517236e-04,-2.607703e-06,2,"]\=""-0092335",North Central,Carmichael Park,2018/03/19 21:30:00,2018/03/19 21:59:59,2018/03/20 17:00:00,5100 Block of Janell,Carmichael,95608,PC 273.5 Inflict Crpl Inj Sp/Cohab
2,6.756477e+06,1.974923e+06,3,03/30-0055028,East,Cordova Meadows,2015/03/06 01:11:00,2015/03/06 01:11:00,2015/03/06 00:00:00,10000 Block of Terra,Rancho Cordova,95670,PC 594(B)(2)(A) Vandalism Less Than $400
3,6.741558e+06,1.960069e+06,4,19-0009846,East,Rosemont Downs,2019/01/09 08:52:59,2019/01/09 09:04:00,2019/01/09 14:19:00,8900 Block of Canberra Dr,Sacramento,95826,PC 459 Burglary
4,6.749211e+06,2.004272e+06,5,1977-0020606,North,Foothill Farms NE,1977/01/01 00:00:00,2001/01/01 00:00:00,2019/08/06 00:00:00,5200 Block of Auburn Blvd,Sacramento,95841,PC 261(A)(1) Rape:Victim Incapable Of Giving C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361999,6.772374e+06,2.000409e+06,362000,2020-8002054,North,Old Fair Oaks,2020/01/17 09:00:00,2020/01/17 15:59:59,2020/01/18 13:57:59,8100 Block of Sunset Ave,Fair Oaks,95628,PC 484G Thft:Use Access Card Data
362001,1.517236e-04,-2.607703e-06,362002,2020-8002082,NaN,NaN,2019/11/20 01:01:00,2019/11/20 23:58:59,2020/01/10 10:44:00,8300 Block of MEDEIROS WAY,SACRAMENTO,95829,PC 368(D) Theft of Elder
362002,6.755575e+06,2.005506e+06,362003,2020-8002107,North,Mercy San Juan,2020/01/21 03:00:00,2020/01/21 03:14:59,2020/01/21 10:28:00,6100 Block of Marwick Way,Carmichael,95608,PC 487 Theft from vehicle > $950
362003,6.764044e+06,1.997386e+06,362004,2-0282708,Northeast,Old Fair Oaks,2012/12/20 13:57:00,2012/12/20 13:57:00,2012/12/20 13:57:00,7400 Block of Windin,Fair Oaks,95628,PC 459 Burglary


In [13]:
# Rename OccuranceZipCode to just ZipCode
preclean_crime_df = preclean_crime_df.rename(columns={"OccurenceZipCode": "ZipCode"})
preclean_crime_df

,X,Y,FID,ActivityNumber,District,Neighborhood,OccurenceStartDate,OccurenceEndDate,ReportDate,OccurenceLocation,OccurenceCity,ZipCode,PrimaryViolation
0,6.765513e+06,1.994405e+06,1,NaN,NaN,Old Fair Oaks,2009/03/26 17:10:00,2009/03/26 17:19:59,2009/03/26 17:39:00,9300 Block of Fair O,Fair Oaks,95628,PC 594(B)(1) Vandalism ($400 Or More)
1,1.517236e-04,-2.607703e-06,2,"]\=""-0092335",North Central,Carmichael Park,2018/03/19 21:30:00,2018/03/19 21:59:59,2018/03/20 17:00:00,5100 Block of Janell,Carmichael,95608,PC 273.5 Inflict Crpl Inj Sp/Cohab
2,6.756477e+06,1.974923e+06,3,03/30-0055028,East,Cordova Meadows,2015/03/06 01:11:00,2015/03/06 01:11:00,2015/03/06 00:00:00,10000 Block of Terra,Rancho Cordova,95670,PC 594(B)(2)(A) Vandalism Less Than $400
3,6.741558e+06,1.960069e+06,4,19-0009846,East,Rosemont Downs,2019/01/09 08:52:59,2019/01/09 09:04:00,2019/01/09 14:19:00,8900 Block of Canberra Dr,Sacramento,95826,PC 459 Burglary
4,6.749211e+06,2.004272e+06,5,1977-0020606,North,Foothill Farms NE,1977/01/01 00:00:00,2001/01/01 00:00:00,2019/08/06 00:00:00,5200 Block of Auburn Blvd,Sacramento,95841,PC 261(A)(1) Rape:Victim Incapable Of Giving C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361999,6.772374e+06,2.000409e+06,362000,2020-8002054,North,Old Fair Oaks,2020/01/17 09:00:00,2020/01/17 15:59:59,2020/01/18 13:57:59,8100 Block of Sunset Ave,Fair Oaks,95628,PC 484G Thft:Use Access Card Data
362001,1.517236e-04,-2.607703e-06,362002,2020-8002082,NaN,NaN,2019/11/20 01:01:00,2019/11/20 23:58:59,2020/01/10 10:44:00,8300 Block of MEDEIROS WAY,SACRAMENTO,95829,PC 368(D) Theft of Elder
362002,6.755575e+06,2.005506e+06,362003,2020-8002107,North,Mercy San Juan,2020/01/21 03:00:00,2020/01/21 03:14:59,2020/01/21 10:28:00,6100 Block of Marwick Way,Carmichael,95608,PC 487 Theft from vehicle > $950
362003,6.764044e+06,1.997386e+06,362004,2-0282708,Northeast,Old Fair Oaks,2012/12/20 13:57:00,2012/12/20 13:57:00,2012/12/20 13:57:00,7400 Block of Windin,Fair Oaks,95628,PC 459 Burglary


In [16]:
# Just Grab the year from the date stamps and make a new column for it
preclean_crime_df["DateTime"] = pd.to_datetime(preclean_crime_df["OccurenceStartDate"])
preclean_crime_df["Year"] = preclean_crime_df["DateTime"].dt.year
preclean_crime_df.head()

,X,Y,FID,ActivityNumber,District,Neighborhood,OccurenceStartDate,OccurenceEndDate,ReportDate,OccurenceLocation,OccurenceCity,ZipCode,PrimaryViolation,DateTime,Year
0,6.765513e+06,1.994405e+06,1,NaN,NaN,Old Fair Oaks,2009/03/26 17:10:00,2009/03/26 17:19:59,2009/03/26 17:39:00,9300 Block of Fair O,Fair Oaks,95628,PC 594(B)(1) Vandalism ($400 Or More),2009-03-26 17:10:00,2009
1,1.517236e-04,-2.607703e-06,2,"]\=""-0092335",North Central,Carmichael Park,2018/03/19 21:30:00,2018/03/19 21:59:59,2018/03/20 17:00:00,5100 Block of Janell,Carmichael,95608,PC 273.5 Inflict Crpl Inj Sp/Cohab,2018-03-19 21:30:00,2018
2,6.756477e+06,1.974923e+06,3,03/30-0055028,East,Cordova Meadows,2015/03/06 01:11:00,2015/03/06 01:11:00,2015/03/06 00:00:00,10000 Block of Terra,Rancho Cordova,95670,PC 594(B)(2)(A) Vandalism Less Than $400,2015-03-06 01:11:00,2015
3,6.741558e+06,1.960069e+06,4,19-0009846,East,Rosemont Downs,2019/01/09 08:52:59,2019/01/09 09:04:00,2019/01/09 14:19:00,8900 Block of Canberra Dr,Sacramento,95826,PC 459 Burglary,2019-01-09 08:52:59,2019
4,6.749211e+06,2.004272e+06,5,1977-0020606,North,Foothill Farms NE,1977/01/01 00:00:00,2001/01/01 00:00:00,2019/08/06 00:00:00,5200 Block of Auburn Blvd,Sacramento,95841,PC 261(A)(1) Rape:Victim Incapable Of Giving C...,1977-01-01 00:00:00,1977


In [19]:
# Just Grab years 2014-2019
preclean_crime_years_filter = preclean_crime_df["Year"].isin([2014, 2015, 2016, 2017, 2018, 2019])
preclean_crime_years_df = preclean_crime_df[preclean_crime_years_filter]
preclean_crime_years_df

,X,Y,FID,ActivityNumber,District,Neighborhood,OccurenceStartDate,OccurenceEndDate,ReportDate,OccurenceLocation,OccurenceCity,ZipCode,PrimaryViolation,DateTime,Year
1,1.517236e-04,-2.607703e-06,2,"]\=""-0092335",North Central,Carmichael Park,2018/03/19 21:30:00,2018/03/19 21:59:59,2018/03/20 17:00:00,5100 Block of Janell,Carmichael,95608,PC 273.5 Inflict Crpl Inj Sp/Cohab,2018-03-19 21:30:00,2018
2,6.756477e+06,1.974923e+06,3,03/30-0055028,East,Cordova Meadows,2015/03/06 01:11:00,2015/03/06 01:11:00,2015/03/06 00:00:00,10000 Block of Terra,Rancho Cordova,95670,PC 594(B)(2)(A) Vandalism Less Than $400,2015-03-06 01:11:00,2015
3,6.741558e+06,1.960069e+06,4,19-0009846,East,Rosemont Downs,2019/01/09 08:52:59,2019/01/09 09:04:00,2019/01/09 14:19:00,8900 Block of Canberra Dr,Sacramento,95826,PC 459 Burglary,2019-01-09 08:52:59,2019
218368,6.753071e+06,1.967571e+06,218369,2013-0289868,East,Lincoln Village,2014/01/01 12:56:00,2014/01/01 01:44:59,2014/01/01 01:44:59,Egmont Way & Viking,Sacramento,95827,VC 23152(A) Dui Alcohol/Drugs,2014-01-01 12:56:00,2014
226870,6.754609e+06,1.968828e+06,226871,2014-0000030,East,Lincoln Village,2014/01/01 00:14:59,2014/01/01 00:14:59,2014/01/01 00:42:00,3300 Block of Explor,Sacramento,95827,PC 246 Shoot At Inhabited Dwelling/Vehicle/Etc,2014-01-01 00:14:59,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361972,6.728830e+06,1.974207e+06,361973,2020-8001802,North Central,Fair Lake Village,2019/12/30 23:00:00,2019/12/31 00:01:00,2020/01/06 16:31:59,900 Block of Howe Ave,Sacramento,95825,PC 530.5(B) Using Another Person's Identifying...,2019-12-30 23:00:00,2019
361979,6.744194e+06,1.973256e+06,361980,2020-8001952,North Central,Wilhaggin,2019/12/21 01:25:00,2019/12/21 01:25:59,2020/01/07 10:40:00,4400 Block of Ashton Dr,Sacramento,95864,PC 484 Theft from building < $950,2019-12-21 01:25:00,2019
361992,6.743273e+06,2.015376e+06,361993,2020-8002039,Northwest,North Highlands,2019/12/10 18:00:00,2019/12/10 18:00:00,2020/01/13 14:52:59,W Elkhorn Blvd & Walerga Rd,Sacramento,95842,PC 484 Theft All Other < $950,2019-12-10 18:00:00,2019
362001,1.517236e-04,-2.607703e-06,362002,2020-8002082,NaN,NaN,2019/11/20 01:01:00,2019/11/20 23:58:59,2020/01/10 10:44:00,8300 Block of MEDEIROS WAY,SACRAMENTO,95829,PC 368(D) Theft of Elder,2019-11-20 01:01:00,2019
